# Developing the theme of few-shots

## Init (imports, load models...)

### Necessary imports

In [3]:
from langchain_community.llms.ollama import Ollama

from langchain_community.vectorstores.pgvector import PGVector
from langchain_community.utilities.sql_database import SQLDatabase

from langchain_community.embeddings.ollama import OllamaEmbeddings

from langchain_core.prompts import FewShotPromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

from langchain.chains.sql_database import query

from langchain.agents import Tool
from langchain import agents

from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit 
from langchain_community.agent_toolkits.sql import base

from langchain.prompts.example_selector import SemanticSimilarityExampleSelector

from langchain.globals import set_verbose
set_verbose(True)

from pswrd import PASSWORD_OF_DB
from pswrd import PASSWORD_FOR_VC_CREATOR

### Load models

In [4]:
llama = Ollama(model="llama2:13b", temperature=0)
embeddings = OllamaEmbeddings(model="llama2:13b")
sqlcoder = Ollama(model='sqlcoder:15b', temperature=0)

### Save examples for few-shot prompt

In [5]:
examples = [
    {
        "input": "How many passengers are in the database?",
        "query": "SELECT COUNT(*) FROM public.\"passenger\";"
    },
    {
        "input": "What are the departure times of all flights?",
        "query": "SELECT \"time_out\" FROM public.\"trip\""
    },
    {
        "input": "What is Jane's place?",
        "query": "SELECT \"place\" FROM public.\"pass_in_trip\" JOIN public.\"passenger\" ON public.\"passenger\".\"id\" = public.\"pass_in_trip\".\"passenger\" WHERE public.\"passenger\".\"name\" = \'John\'"
    },
    {
        "input": "Give me all information about airlines",
        "query": "SELECT * FROM public.\"company\""
    },
    {
        "input": "Show me all the trips that are flying out today",
        "query": "SELECT * FROM public.\"trip\"\nWHERE EXTRACT(DAY FROM NOW()) = EXTRACT(DAY FROM \"time_out\")"
    },
    {
        "input": "Which planes depart from Washington?",
        "query": "SELECT \"plane\" FROM public.\"trip\" WHERE \"town_from\" = \'Washington\'"
    },
    {
        "input": "Print out the names of all the planes", 
        "query": "SELECT \"plane\" FROM public.\"trip\""
    },
    {
        "input": "How many people fly on Airbus?",
        "query": "SELECT COUNT(*) FROM public.\"pass_in_trip\" AS paip JOIN public.\"trip\" ON trip.\"id\" = paip.\"trip\" WHERE trip.\"plane\" = \'Airbus\'"
    },
]

### Create prompt

In [6]:
prefix = \
"""You are a PostreSQL expert. Given an input question, first create {top_k} syntactically correct PostreSQL query to run, then look at the results and take most correct.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per PostreSQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (\") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".
Below are a number of examples of questions and their corresponding SQL queries."""

suffix = \
"""Only use the following tables:
{table_info}

Use the following format:

User input: {input}
SQL query: """

few_shots_prompt = PromptTemplate.from_template(
    "User input: {input}\nSQL query: {query}"
)

few_shots_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=few_shots_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input", "table_info", "top_k"],
)

### Connect to DB with Readonly role

In [7]:
CONNECTION_STRING = PGVector.connection_string_from_db_params(
    driver="psycopg2",
    host="localhost",
    port=5433,
    database="llama-test-2",
    user="seq2sql_llama2_rag",
    password=PASSWORD_OF_DB
)

In [8]:
db = SQLDatabase.from_uri(CONNECTION_STRING)

In [9]:
db.run("SET ROLE pg_read_all_data");

#### Check connection

In [10]:
db.run("select * from passenger")

"[(1, 'John'), (2, 'James'), (3, 'Poul'), (4, 'Christofer'), (5, 'Superman')]"

## Simple few-shots by using `create_sql_query_chain`

In [11]:
few_shots_chain = query.create_sql_query_chain(llm=llama, db=db, prompt=few_shots_prompt)

In [12]:
res = few_shots_chain.invoke({"question": "Select the names of all the people who are in the airline database"})
print(res)

User input: Select the names of all the people who are in the airline database

SQL query: SELECT name FROM passenger;

Explanation: To retrieve the names of all the people in the airline database, we can simply select the "name" column from the "passenger" table. This will return all the names of the passengers in the database.


In [13]:
db.run("SELECT name FROM passenger")

"[('John',), ('James',), ('Poul',), ('Christofer',), ('Superman',)]"

In [14]:
res = few_shots_chain.invoke({"question": "Print the names of all airlines"})
print(res)

User input: Print the names of all airlines

SQL query: SELECT name FROM company;

Explanation: The "company" table contains the list of all airlines, so we can simply select the "name" column from this table to get the desired output.


In [15]:
db.run("SELECT name FROM company")

"[('American Airlines',)]"

In [16]:
res = few_shots_chain.invoke({"question": "Print the names of people that end in \"man\""})
print(res)

User input: Print the names of people that end in "man"

SQL query: SELECT name FROM passenger WHERE name LIKE '%man';

This query uses the LIKE operator to search for names that end with "man". The % wildcard character is used to match any sequence of zero or more characters before the "man" pattern.


In [17]:
db.run("SELECT name FROM passenger WHERE name LIKE \'%man\'")

"[('Superman',)]"